1. Make diffrential equations with repspect to time
    1. Assuming no intial conditions
    2. Make a s-domain to t domain converter?
2. solve diffrential equations
3. plot the equations

In [7]:
#| default_exp parsing
import numpy as np
import sympy as sp
import scipy
#from scipy.signal import inverse_lapace_transfrom 

In [8]:
#| export
class MoreFreqError(Exception):
    pass

def parse_circuit(netlist):
    #read the file and load it contents to a list
    try:
        with open(netlist,'r') as f:
            para = f.read().splitlines()
    except FileNotFoundError:
        print('Make sure the path of the .netlist or .txt file is same as this notebook')

    #find .circuit and .ac or .end and remove others unneccesaary things
    try:
        index1 = para.index('.circuit')
        index2 = para.index('.end')
        string = re.findall('\.ac.*', '\n'.join(para))

        #update the content to have data between .circuit and .end using string slicing method
        para = para[index1:index2+1]

        #update content to have data regarding frequencies
        para.extend(string)

        #to check if there is only one frequency
        w = []
        if len(string)>=1:
            for line in string:
                w_st = line.split()
                w.append(int(w_st[2]))
        try:
            if max(w) != min(w):
                raise MoreFreqError
        except MoreFreqError:
            print('More than one AC frequencys')
        except ValueError:
            pass
    
    #remove comments from the paragraph or the strings after '#'  
        filtered = []
        for line in para:
            #l = re.sub(r'#.*','',line)
            l = line.split('#')
            filtered.append(l[0])
        #the proccesed content is filtered
        return filtered
    except ValueError:
        print('Invalid Netlist')
    #find .ac in the content as .ac comes after .end

    

In [9]:
x = parse_circuit('ckt6.txt')
x

['.circuit',
 'V1 n1 0 5',
 'R1 n1 n2 0.1',
 'L1 n2 n3 0.2',
 'C1 n3 0 0.4',
 '.end']

In [10]:
#import required libraries
import re
#make a custom error for having more than one frequencies in netlist
class MoreFreqError(Exception):
    pass

#netlist = input('What is the file name? ')  

netlist = 'ckt6.txt'  #<---------uncommet this to hardcode file name

#read the file and load it contents to a list
try:
    with open(netlist,'r') as f:
        para = f.read().splitlines()
except FileNotFoundError:
    print('Make sure the path of the .netlist or .txt file is same as this notebook')
    
#find .circuit and .ac or .end and remove others unneccesaary things
try:
    index1 = para.index('.circuit')
    index2 = para.index('.end')
    string = re.findall('\.ac.*', '\n'.join(para))

    #update the content to have data between .circuit and .end using string slicing method
    para = para[index1:index2+1]

    #update content to have data regarding frequencies
    para.extend(string)

    #to check if there is only one frequency
    w = []
    if len(string)>=1:
        for line in string:
            w_st = line.split()
            w.append(int(w_st[2]))
    try:
        if max(w) != min(w):
            raise MoreFreqError
    except MoreFreqError:
        print('More than one AC frequencys')
    except ValueError:
        pass
    

    #remove comments from the paragraph or the strings after '#'  
    filtered = []
    for line in para:
        #l = re.sub(r'#.*','',line)
        l = line.split('#')
        filtered.append(l[0])
    #the proccesed content is filtered
    print(filtered)
except ValueError:
    print('Invalid Netlist')
#find .ac in the content as .ac comes after .end



['.circuit', 'V1 n1 0 5', 'R1 n1 n2 0.1', 'L1 n2 n3 0.2', 'C1 n3 0 0.4', '.end']


In [11]:
#| export

#delete .circuit and .end and return if the circuit is ac or dc and if ac then
# collect operating frequency
def delete_headings(circ):
    data = [' '.join(x.split()) for x in circ]
    mode = circ[-1]
    data = [n for n in data if not n.startswith('.')] 
    if mode == '.end':
        return (0,'dc',data)
    else:
        k = mode.split()
        return (k[2],'ac', data)
    
    
def process_net(netlist):
    (w0,mode,y) = delete_headings(netlist)
    
    w0 = int(w0)
    flag_ac = 0
    flag_dc = 0
    
    #Making useful data list
    data = []
    try:
        for line in y:
            line = line.split(' ')
            if line[1] == 'GND':       # Changing the GND node to 0 node for calculation purposes
                line[1] = '0'
            if line[2] == 'GND':
                line[2] = '0'
            #Handling for non nuumeric nodes   
            if line[1][0] == 'n':
                line[1] = line[1][1:]
            if line[2][0] == 'n':
                line[2] = line[2][1:]

            if line[3] == 'dc' or line[3] == 'ac': #Could have use mode 
                if line[3] == 'dc':
                    flag_dc = 1
                else:
                    flag_ac = 1
                del line[3]                        # delete 'ac' or 'dc' strings
            x = ' '.join(line)
            data.append(x)
        return data,flag_ac,flag_dc,w0
    except:
        print('Invalid Netlist')   

In [17]:
y,f_1,f_2,w = process_net(x)
ls = []
for line in y:
    ls.append(line.split())
ls

[['V1', '1', '0', '5'],
 ['R1', '1', '2', '0.1'],
 ['L1', '2', '3', '0.2'],
 ['C1', '3', '0', '0.4']]

In [ ]:
y[0].split()

In [ ]:
#intializing variables
rlc_count = 0
v_count = 0
i_count = 0

# if flag_ac and flag_dc:    #Cathing case of AC DC mixed circuit
#     print('***AC and DC mixed circuit found***')


#delete .circuit and .end and return if the circuit is ac or dc and if ac then
# collect operating frequency
def delete_headings(circ):
    data = [' '.join(x.split()) for x in circ]
    mode = circ[-1]
    data = [n for n in data if not n.startswith('.')] 
    if mode == '.end':
        return (0,'dc',data)
    else:
        k = mode.split()
        return (k[2],'ac', data)
    
(w0,mode,y) = delete_headings(filtered)
#frequency of circuit 
w0 = int(w0)
flag_ac = 0
flag_dc = 0
#Making useful data list
data = []
try:
    for line in y:
        line = line.split(' ')
        if line[1] == 'GND':       # Changing the GND node to 0 node for calculation purposes
            line[1] = '0'
        if line[2] == 'GND':
            line[2] = '0'
        #Handling for non nuumeric nodes   
        if line[1][0] == 'n':
            line[1] = line[1][1:]
        if line[2][0] == 'n':
            line[2] = line[2][1:]

        if line[3] == 'dc' or line[3] == 'ac': #Could have use mode 
            if line[3] == 'dc':
                flag_dc = 1
            else:
                flag_ac = 1
            del line[3]                        # delete 'ac' or 'dc' strings
        x = ' '.join(line)
        data.append(x)
except:
    print('Invalid Netlist')
data   

In [ ]:
data[0].split()

In [ ]:
#| export



#making a loading function that takes values from data 
#and stores to dictionary for passive elements
def load_rlc(line_number):
    line = data[line_number].split()
    data_dic['element'] += [line[0]]
    data_dic['+node'] += [int(line[1])]
    data_dic['-node'] += [int(line[2])]
    data_dic['value'] += [float(line[3])]
    if line[0][0] == 'R':                #for ac circuit
        data_dic['phase'] += [0.0]
    elif line[0][0] == 'L':
        data_dic['phase'] += [np.pi/2]
    elif line[0][0] == 'C':
        data_dic['phase'] += [np.pi/2]
        
        
#making a loading function that takes values from data 
#and stores to dictionary for active elements
def load_sources(line_number):
    line = data[line_number].split()
    if mode == 'dc':
        data_dic['element'] += [line[0]]
        data_dic['+node'] += [int(line[1])]
        data_dic['-node'] += [int(line[2])]
        data_dic['value'] += [float(line[3])]
        data_dic['phase'] += [0.0]
    elif mode == 'ac':
            data_dic['element'] += [line[0]]
            data_dic['+node'] += [int(line[1])]
            data_dic['-node'] += [int(line[2])]
            data_dic['value'] += [float(line[3])]
            try:                                     #for ac mode
                data_dic['phase'] += [float(line[4])]
            except IndexError:
                data_dic['phase'] += [0.0]
            
    else:
        return('Unknown mode')
        
    
        
    
# load_rlc(0)
# load_rlc(1)
# load_rlc(2)

In [ ]:
# number of components and branches
line_count = len(data) 
branch_count = 0     #no need for this as line_count = branch_count but done anyways

#counting number of passive elements rlc and active elemets V ,I
for i in range(line_count):
    comp = data[i][0]      #the 0th index shows name of component
    value_count = len(data[i].split()) #value count is a nested list containg a list
                                       #that contains name of component
                                       #postive node, negative node and value of 
                                       #the component
    if (comp == 'R') or (comp == 'L') or (comp == 'C'): #add capatlize function
        if value_count != 4:           # handling errors
            print("Error in netlist")
            print('Error in passive elements RLC')
        rlc_count += 1
        branch_count += 1
    elif comp == 'V':
        if value_count != 5 and value_count != 4:
            print("Error in netlist ")
            print('Error in Voltage source/node')
        v_count += 1
        branch_count += 1
    elif comp == 'I':
        if value_count != 4 and value_count != 5:
            print("Error in netlist ")
            print('Error in Current source/branch')
        i_count += 1
        branch_count += 1
    else:
        print('Unknow element found in netlist')
         


In [ ]:
#| export
def Error(data):
        # number of components and branches
    rlc_count = 0
    v_count = 0
    i_count = 0
    branch_count = 0     #no need for this as line_count = branch_count but done anyways
    
    line_count = len(data)

    #counting number of passive elements rlc and active elemets V ,I
    for i in range(line_count):
        comp = data[i][0]      #the 0th index shows name of component
        value_count = len(data[i].split()) #value count is a nested list containg a list
                                           #that contains name of component
                                           #postive node, negative node and value of 
                                           #the component
        if (comp == 'R') or (comp == 'L') or (comp == 'C'): #add capatlize function
            if value_count != 4:           # handling errors
                print("Error in netlist")
                print('Error in passive elements RLC')
            rlc_count += 1
            branch_count += 1
        elif comp == 'V':
            if value_count != 5 and value_count != 4:
                print("Error in netlist ")
                print('Error in Voltage source/node')
            v_count += 1
            branch_count += 1
        elif comp == 'I':
            if value_count != 4 and value_count != 5:
                print("Error in netlist ")
                print('Error in Current source/branch')
            i_count += 1
            branch_count += 1
        else:
            print('Unknow element found in netlist')
            
    return rlc_count,v_count,i_count,branch_count

In [ ]:
rlc,v,i,bc = Error(y)

In [ ]:
#| export
def count_nodes():
    n = [([0]*(line_count+1)) for i in range(1)]
    for i in range(line_count - 1):
        n[0][data_dic['+node'][i]] = data_dic['+node'][i]
        n[0][data_dic['-node'][i]] = data_dic['-node'][i]
        #largetst node
    if max(data_dic['-node']) > max(data_dic['+node']):
        largest = max(data_dic['-node'])
    else:
        largest =  max(data_dic['+node'])
        
    # check for unfilled elements, skip node 0
    for i in range(1,largest):
        if n[0][i] == 0:
            print('Error in node order')
    return largest

In [ ]:
#| export
#load data into dictionary
def load_data(data,data_dic):
    line_count = len(data)
    #using data to make a dictionary that can help in extracting data easily
    for i in range(line_count):
        comp = data[i][0]
        if (comp == 'R') or (comp == 'L') or (comp == 'C'):
            load_rlc(i)
        elif (comp == 'V') or (comp == 'I'):
            load_sources(i)
        else:
            print('Unknown Elelment Error')

    # count number of nodes
    #num_nodes = count_nodes() #maximum node number in the circui

In [ ]:
data_dic = {'element':[],'+node':[],'-node':[],'value':[],'phase':[]}
load_data(y,data_dic)
data_dic

In [ ]:
#export to py file.
import nbdev
nbdev.export.nb_export('Bonus.ipynb', './')
print('Export successful')

In [ ]:


#make matricies for calculations
#V matrix to store variable names of voltage nodes
V = sp.zeros(num_nodes,1)
#I matrix to store values of independent current sources
# I = np.zeros((num_nodes,1),dtype='complex_')
I =  sp.zeros(num_nodes,1)
#G matrix as in conductance matrix for the equations
G = np.zeros((num_nodes,num_nodes),dtype='complex_')
GS = sp.zeros(num_nodes,num_nodes)
s = sp.Symbol('s')

# count the number of element types that affect the size of the B, C, D, E and J arrays
k = v_count
if k != 0:
    B = sp.zeros(num_nodes,k)
    C = sp.zeros(k,num_nodes)
    D = sp.zeros(k,k)
    E = sp.zeros(k,1)
    J = sp.zeros(k,1)
k

In [ ]:
gs = 0
for i in range(branch_count):  
    n1 = data_dic['+node'][i] #first node is + 
    n2 = data_dic['-node'][i] #second node is - 
    # iterate through each element and save them in temporary variable g
    # then appy the rules to make G matrix
    comp = data_dic['element'][i][0]   #The first letter shows element type
    if comp == 'R':
        gs = 1/sp.sympify(data_dic['element'][i])
        g = 1/data_dic['value'][i] 
    if comp == 'L':
        gs = 1/(s*sp.sympify(data_dic['element'][i]))
        if mode == 'ac':
            g = 1/(1j*w0*data_dic['value'][i]) 
        else:
            g = np.inf                          #for dc L behaves as close circuit in steady state
    if comp == 'C':
        gs = s*sp.sympify(data_dic['element'][i])
        if mode == 'ac':
            g = 1j*w0*data_dic['value'][i]     
        else:
            g = 0                               #for dc C behaves as open circuit in steady state

    if (comp == 'R') or (comp == 'L') or (comp == 'C'):
        # If neither side of the element is connected to ground
        # then subtract it from appropriate location in matrix.
        if (n1 != 0) and (n2 != 0):
            G[n1-1,n2-1] += -g
            G[n2-1,n1-1] += -g
            
            GS[n1-1,n2-1] += -gs
            GS[n2-1,n1-1] += -gs

        # If node 1 is connected to ground, add element to diagonal of matrix
        if n1 != 0:
            G[n1-1,n1-1] += g
            GS[n1-1,n1-1] += gs

        # same for for node 2
        if n2 != 0:
            G[n2-1,n2-1] += g
            GS[n2-1,n2-1] += gs


GS

In [ ]:
# generate the B Matrix
# loop through all the branches and process independent voltage sources
sources = 0   # count source number
for i in range(branch_count):
    n1 = data_dic['+node'][i]
    n2 = data_dic['-node'][i]
    # process all the independent voltage sources
    #x = df.loc[i,'element'][0]   #get 1st letter of element name
    comp = data_dic['element'][i][0]
    if comp == 'V':
        if v_count > 1:
            if n1 != 0:
                B[n1-1][sources] = 1
            if n2 != 0:
                B[n2-1][sources] = -1
            sources += 1   #increment source count
        else:
            if n1 != 0:
                B[n1-1] = -1
            if n2 != 0:
                B[n2-1] = 1
        
B

In [ ]:
source = 0   # count source number
for i in range(branch_count):
    #n1 = df.loc[i,'p node']
    n1 = data_dic['+node'][i]
    n2 = data_dic['-node'][i]
    # process all the independent voltage sources
    #x = df.loc[i,'element'][0]   #get 1st letter of element name
    comp = data_dic['element'][i][0]
    if comp == 'V':
        if v_count > 1:
            if n1 != 0:
                C[source][n1-1] = 1
            if n2 != 0:
                C[source][n2-1] = -1
            source += 1   #increment source count
        else:
            if n1 != 0:
                C[n1-1] = -1
            if n2 != 0:
                C[n2-1] = 1
C 

In [ ]:
# Current matrix containg current at each node
for i in range(branch_count):
    #n1 = df.loc[i,'p node']
    n1 = data_dic['+node'][i]
    n2 = data_dic['-node'][i]
    # process all the passive elements, save conductance to temp value
    comp = data_dic['element'][i][0]
    if comp == 'I':
        #g = data_dic['element'][i]
        g = data_dic['value'][i]*np.exp(1j*data_dic['phase'][i]) # For AC in, case dc phase = 0
        #g = data_dic['value'][i]
        # sum the current into each node
        if n1 != 0:
            I[n1-1] = I[n1-1] - g
        if n2 != 0:
            I[n2-1] = I[n2-1] + g
            
I

In [ ]:
for i in range(num_nodes):
    V[i] = sp.sympify('v{:d}'.format(i+1))

V  

In [ ]:
# matrix J for current through voltage sources
sources = 0   # count source number
for i in range(branch_count):
    # process all the passive elements
    comp = data_dic['element'][i][0]  
    if comp == 'V':
        #J[sorces] = sympify('I_{:s}'.format(df.loc[i,'element']))
        J[sources] = 'I_{:s}'.format(data_dic['element'][i])
        sources += 1

J  

In [ ]:
# generate the E matrix
sn = 0   # count source number
for i in range(branch_count):
    # process all the passive elements
    x = data_dic['element'][i][0]   #get 1st letter of element name
    if x == 'V':
        E[sn] = sp.sympify(data_dic['element'][i])
        sn += 1

E # display the E matrix

In [ ]:
Z = sp.Matrix(I[:] + E[:])
Z  # display the Z matrix

In [ ]:
X = sp.Matrix(V[:] + J[:])
type(X) # display the X matrix

In [ ]:
n = num_nodes
m = v_count
A = sp.zeros(m+n,m+n)
for i in range(n):
    for j in range(n):
        A[i,j] = GS[i,j]

if v_count > 1:
    for i in range(n):
        for j in range(m):
            A[i,n+j] = B[i,j]
            A[n+j,i] = C[j,i]
else:
    for i in range(n):
        A[i,n] = B[i]
        A[n,i] = C[i]

A  # display the A matrix

In [ ]:
n = num_nodes
# m = k
eq_temp = 0  # temporary equation used to build up the equation
equ = []  #initialize the array to hold the equations
for i in range(n+k):
    for j in range(n+k):
#         print(i,j)
        eq_temp += A[i,j]*X[j]
        
    qu = sp.Eq(eq_temp,Z[i])
    
    equ.append(qu)
    eq_temp = 0

sp.Array(equ)  # display the equations

In [ ]:
#To the time domain

equ[0]

In [ ]:
A*X-Z

In [ ]:
X

In [ ]:
solutions = sp.solve(A*X-Z,X)

In [ ]:
solutions

In [ ]:
sol=solutions

In [ ]:
sol.keys()

In [ ]:
sus = sol[sp.Symbol('v2')].subs({sp.Symbol('C1'):0.4,sp.Symbol('R1'):0.1,sp.Symbol('L1'):0.2,sp.Symbol('V1'):10})
sus

In [ ]:
t = sp.Symbol('t')
def invL(F):
    return sp.inverse_laplace_transform(F, s, t)

In [ ]:
k = invL(sus)
kreal,kimag = k.as_real_imag()
kreal

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
time = np.linspace(0,15,100)


In [ ]:
y = []
for tt in time:
    y.append(kreal.subs({t:tt}))

In [ ]:
plt.plot(time,y)

In [ ]:
y2 = []
y2.extend(y[1:])

In [ ]:
plt.plot(time[1:],y2)

In [ ]:
y[0].subs({sp.DiracDelta[0.0]:1})

In [ ]:
# Create a list of indices to remove
indices_to_remove = [i for i, x in enumerate(y) if sp.DiracDelta(0.0) in x.args]

# Remove the elements containing DiracDelta(0.0) from both lists
y = [x for i, x in enumerate(y) if i not in indices_to_remove]
time = [x for i, x in enumerate(time) if i not in indices_to_remove]


In [ ]:
y[0] - 10*sp.DiracDelta(0.0)

In [ ]:
new_list = [elem for elem in y if not (isinstance(elem, sp.DiracDelta) and elem.subs(x, 0.0) != 0.0)]

In [ ]:
print(new_list)

In [ ]:
if sp.DiracDelta(0.0) in y[0]:
    print('yes')

In [ ]:
type(y[1])

In [ ]:
x = []
z = []
yt = list(zip(y,time))
for i,tt in yt:
    try:
        x.append(float(i))
        z.append(float(time))
    except:
        pass

In [ ]:
x